In [ ]:
!pip install -q -U transformers langchain bitsandbytes qdrant-client
!pip install -q -U sentence-transformers accelerate unstructured


In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
import os
import torch
from operator import itemgetter
from transformers import BitsAndBytesConfig
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from langchain.vectorstores import Qdrant
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_community.document_loaders import DirectoryLoader
from langchain.embeddings import HuggingFaceEmbeddings
file_path = '/content/drive/MyDrive/Colab Notebooks/data/Object_casedocs_500/'


In [ ]:
# preparing config for quantizing the model into 4 bits

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)

# load the tokenizer and the quantized mistral model
model_id = "mistralai/Mistral-7B-Instruct-v0.2"
model_4bit = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto",quantization_config=quantization_config,)
tokenizer = AutoTokenizer.from_pretrained(model_id)

# using HuggingFace's pipeline
pipeline = pipeline(
        "text-generation",
        model=model_4bit,
        tokenizer=tokenizer,
        use_cache=True,
        device_map="auto",
        max_new_tokens=5000,
        do_sample=True,
        top_k=3,
        temperature = 0.01,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.eos_token_id,
)



In [ ]:
model = HuggingFacePipeline(pipeline=pipeline)

In [ ]:
# load the legal case documents
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=20,
    length_function=len,
    is_separator_regex=False,
)

loader = DirectoryLoader(file_path)
docs = loader.load()
print(len(docs))



In [ ]:
docs = text_splitter.split_documents(docs)
docs

In [ ]:
# define the embedding model
emb_model = "sentence-transformers/all-MiniLM-L6-v2"
embeddings = HuggingFaceEmbeddings(
    model_name=emb_model,
    cache_folder=os.getenv('SENTENCE_TRANSFORMERS_HOME'))

In [ ]:
qdrant_collection = Qdrant.from_documents(
docs,
embeddings,
location=":memory:", # Local mode with in-memory storage only
collection_name="it_resumes",
)
# construct a retriever on top of the vector store
qdrant_retriever = qdrant_collection.as_retriever()

In [ ]:
# let's try a query and see the how its retrieved from the qdrant vector database
qdrant_retriever.invoke('Cite me a dispute related to electricity board tender')

In [ ]:
template = """<s>[INST] You are a helpful, respectful and honest legal assistant.
Your task is to assist lawyers in legal case discovery.
Answer the question below from context below.
{context}
{question} [/INST] </s>
"""

prompt = ChatPromptTemplate.from_template(template)


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": qdrant_retriever| format_docs, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)


In [ ]:
rag_chain.invoke("Cite me a dispute related to electricity board tender")


"In this dispute, the Chhattisgarh State Electricity Board (CSEB) invited tenders for the Designing, Engineering, Testing, Supply, Erection & Commission of HEA Ignition system at Hasedeo Thermal Power Station (Korba West). The respondent, M/s Control Electronics India (CEI), submitted an application for the tender documents but it was rejected due to incomplete documents, specifically the non-submission of documentary evidence of past performance and experience. The respondent then complained against the appellant for not issuing the tender documents.\n\nHowever, the present case is not about the initial rejection of the respondent's application. Instead, it revolves around the respondent's allegations that the records were fabricated and the tender document was not opened and returned in furtherance of official duties by the appellant.\n\nThe Court has carefully examined the impugned advertisement issued by the Nagar Nigam and found no illegality in it. The Court has also considered t

In [ ]:
rag_chain.invoke("In the dispute related to electricity board tender, what was the outcome?")


"In this dispute, the respondent had filed a civil suit challenging the decision of the Electricity Board in returning his tender documents due to non-compliance with the pre-qualifying conditions. However, he withdrew the suit, leading to its dismissal for non-prosecution. The respondent's attempt to challenge the decision of the Tender Committee in not considering his tender was unfaulted due to the constructive res judicata effect of the withdrawn suit. The tender of the respondent was rejected due to the defects in the scanning system supplied by him, which adversely affected the generation at Patratu Thermal Power Station. The Tender Committee sought expert opinions and rejected the respondent's tender based on the reports received. The allegations of fabricating records made by the respondent were considered mischievous and an afterthought. The appellant, R.C. Jain, was deputed to verify the claim of the respondent, and he reported that the works carried out by the respondent at 

Now one may argue that the above query is same as the one we passed onto the qdrant retriever, and if so, what is exactly the utility of the LLM here?

In [ ]:
from langchain_core.runnables import RunnableParallel

rag_chain_from_docs = (
    RunnablePassthrough.assign(context=(lambda x: format_docs(x["context"])))
    | prompt
    | model
    | StrOutputParser()
)

rag_chain_with_source = RunnableParallel(
    {"context": qdrant_retriever, "question": RunnablePassthrough()}
).assign(answer=rag_chain_from_docs)

rag_chain_with_source.invoke("Cite me a dispute related to electricity board tender")

{'context': [Document(page_content="The Chhattisgarh State Electricity Board (for short 'the CSEB') issued an advertisement inviting tender (NIT) bearing No. T- 136/2004 dated 02.06.2004 for its work at Hasedeo Thermal Power Station (Korba West) towards Designing, Engineering, Testing, Supply, Erection & Commission of HEA Ignition system. The applications received there under were required to be processed in three stages successively namely; Part-I (EMD); Part-II (Techno- Commercial Criteria) and Part III (Price Bid). The respondent herein submitted an application on 26.08.2004 as Chief Executive Officer of M/s Control Electronics India (CEI) requesting for Tender Document. The application was rejected on the ground that it was accompanied by incomplete documents i.e. non-submission of documentary evidence of past performance and experience of the respondent. The respondent made a complaint dated 06.09.2004 against appellant No. 3 herein alleging that the Tender Documents were not issu